In [1]:
!nvidia-smi

Sun Dec  8 22:13:07 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install transformers

     |████████████████████████████████| 368kB 10.1MB/s 
     |████████████████████████████████| 860kB 77.2MB/s 
     |████████████████████████████████| 645kB 49.5MB/s 
     |████████████████████████████████| 1.0MB 56.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=5e55acaefc4659da67b3dfac48fb9fe5431e00ab2976cd0c2491ca25fae990d6
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import os
import sys
import pickle
import torch
import numpy as np
import torch.optim as optim
# from transformers import BertConfig, BertModel, BertForMaskedLM
from transformers import GPT2Config, GPT2Model, GPT2LMHeadModel

from IPython.display import clear_output

In [5]:
# custom parameters for GPT2 model
vocab_size = 21
max_position_embeddings = 20 # 1024
n_ctw = max_position_embeddings # 1024
n_embd = 16 # 768
n_layer = 12 # 12
n_head = 8 # 12
resid_pdrop = 0.1 # 0.1
embd_pdrop = 0.1 # 0.1
attn_pdrop = 0.1 # 0.1
layer_norm_epsilon = 1e-5 # 1e-5


config = GPT2Config(vocab_size_or_config_json_file=vocab_size,
                    n_positions=max_position_embeddings,
                    n_ctw=n_ctw,
                    n_embd=n_embd,
                    n_layer=n_layer,
                    n_head=n_head, 
                    resid_pdrop=resid_pdrop,
                    embd_pdrop=embd_pdrop,
                    attn_pdrop=attn_pdrop,
                    layer_norm_epsilon=layer_norm_epsilon)

model = GPT2LMHeadModel(config)

# print(model)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21, 16)
    (wpe): Embedding(20, 16)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((16,

In [0]:
# load data
d = "drive/My Drive/Colab Notebooks/smaug/data"
shingle_path = os.path.join(d, "ecoli_MG1655_shingles_length20_overlap10.npy")

with open(shingle_path, 'rb') as f:
    ecoli_shingles = np.load(shingle_path)#[:2056]

In [0]:
# pass to GPU
tokens_tensor = torch.tensor(ecoli_shingles).to('cuda')
dtrain = tokens_tensor

In [0]:
optimizer = optim.AdamW(model.parameters())

In [0]:
def savemodel():
    # save model
    # Load the Drive helper and mount
    from google.colab import drive

    # This will prompt for authorization.
    drive.mount('/content/drive')

    modeldir = "drive/My Drive/Colab Notebooks/smaug/data/models/ecoli_trivial_length20_overlap10"
    model.save_pretrained(modeldir)
    print(os.listdir(modeldir))

In [0]:
# train model
model.train()

batch_size = 2**9

optimizer.zero_grad()
np.random.seed(42424)
for i in range(int(1e20)):
    if i%500==10:
        savemodel()
    optimizer.zero_grad()

    select_idx = np.random.randint(0, len(dtrain), batch_size)

#     input_ids = dtrain[select_idx].unsqueeze(0) # singleton
    input_ids = dtrain[select_idx]
    
    
    outputs = model(input_ids, labels=input_ids)
    loss, prediction_scores = outputs[:2]
    
    loss.backward()
    optimizer.step()
    
    clear_output(wait=True)
    print("Loss:", loss.item())

    for k in range(0, 5):
#         print(i, torch.argmax(prediction_scores[0,k-1]).item(), input_ids[0,k].item(), "\t", loss.item()) #TODO figure out why GPT2 only offsets sometimes
        print(i, "\t", torch.argmax(prediction_scores[10,k-1]).item(), "\t", input_ids[10,k].item(), "\t")
#         print(prediction_scores[0])
    


Loss: 2.838357925415039
40468 	 1 	 8 	
40468 	 1 	 19 	
40468 	 1 	 3 	
40468 	 1 	 17 	
40468 	 14 	 8 	
